<a href="https://colab.research.google.com/github/koushik-ace/NLP/blob/main/Assignment_6_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [22]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("spsayakpaul/arxiv-paper-abstracts")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/spsayakpaul/arxiv-paper-abstracts/versions/2


In [ ]:
import pandas as pd

df = pd.read_csv(f'{path}/arxiv_data.csv')
df.head()

,titles,summaries,terms
0,Survey on Semantic Stereo Matching / Semantic ...,Stereo matching is one of the widely used tech...,"['cs.CV', 'cs.LG']"
1,FUTURE-AI: Guiding Principles and Consensus Re...,The recent advancements in artificial intellig...,"['cs.CV', 'cs.AI', 'cs.LG']"
2,Enforcing Mutual Consistency of Hard Regions f...,"In this paper, we proposed a novel mutual cons...","['cs.CV', 'cs.AI']"
3,Parameter Decoupling Strategy for Semi-supervi...,Consistency training has proven to be an advan...,['cs.CV']
4,Background-Foreground Segmentation for Interio...,"To ensure safety in automated driving, the cor...","['cs.CV', 'cs.LG']"


## Prepare Corpus



In [ ]:
corpus = df['summaries']
print(corpus.head())

0    Stereo matching is one of the widely used tech...
1    The recent advancements in artificial intellig...
2    In this paper, we proposed a novel mutual cons...
3    Consistency training has proven to be an advan...
4    To ensure safety in automated driving, the cor...
Name: summaries, dtype: object


In [ ]:
import re

def clean_text(text):
    text = text.lower() # Convert to lowercase
    text = re.sub(r'\d+', '', text) # Remove numbers
    text = re.sub(r'[\W_]+', ' ', text) # Remove punctuation and special characters, replace with space
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces and strip leading/trailing spaces
    return text

cleaned_corpus = corpus.apply(clean_text)
print(cleaned_corpus.head())


0    stereo matching is one of the widely used tech...
1    the recent advancements in artificial intellig...
2    in this paper we proposed a novel mutual consi...
3    consistency training has proven to be an advan...
4    to ensure safety in automated driving the corr...
Name: summaries, dtype: object


In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab') # Added to resolve LookupError

stop_words = set(stopwords.words('english'))

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

tokenized_corpus = cleaned_corpus.apply(tokenize_and_remove_stopwords)
print(tokenized_corpus.head())

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


0    [stereo, matching, one, widely, used, techniqu...
1    [recent, advancements, artificial, intelligenc...
2    [paper, proposed, novel, mutual, consistency, ...
3    [consistency, training, proven, advanced, semi...
4    [ensure, safety, automated, driving, correct, ...
Name: summaries, dtype: object


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer

nltk.download('wordnet')
nltk.download('omw-1.4') # Required for WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    lemmas = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmas

lemmatized_corpus = tokenized_corpus.apply(lemmatize_tokens)
print(lemmatized_corpus.head())

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


0    [stereo, matching, one, widely, used, techniqu...
1    [recent, advancement, artificial, intelligence...
2    [paper, proposed, novel, mutual, consistency, ...
3    [consistency, training, proven, advanced, semi...
4    [ensure, safety, automated, driving, correct, ...
Name: summaries, dtype: object


In [ ]:
import sys
!{sys.executable} -m pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 62.7 MB/s eta 0:00:00


In [ ]:
from gensim import corpora

# Create a dictionary from the lemmatized corpus
dictionary = corpora.Dictionary(lemmatized_corpus)

# Filter out words that appear in less than no_below documents
# or more than no_above fraction of total documents
# and keep only the most common keep_n words
dictionary.filter_extremes(no_below=5, no_above=0.5, keep_n=100000)

# Create a Bag-of-Words corpus
bow_corpus = [dictionary.doc2bow(doc) for doc in lemmatized_corpus]

print("Number of unique tokens in dictionary:", len(dictionary))
print("First 5 entries of the BoW corpus:")
for i, doc in enumerate(bow_corpus[:5]):
    print(f"Document {i}: {doc}")

Number of unique tokens in dictionary: 15487
First 5 entries of the BoW corpus:
Document 0: [(0, 1), (1, 1), (2, 1), (3, 1), (4, 2), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 2), (29, 2), (30, 4), (31, 1), (32, 1), (33, 1), (34, 1), (35, 2), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 2), (49, 2), (50, 1), (51, 1), (52, 2), (53, 1), (54, 5), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 2), (61, 1)]
Document 1: [(4, 1), (16, 2), (23, 3), (29, 1), (37, 1), (41, 1), (42, 1), (47, 1), (48, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 8), (67, 1), (68, 1), (69, 1), (70, 1), (71, 2), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 3), (78, 1), (79, 1), (80, 1), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 

In [ ]:
from gensim.models import LdaModel

# Set the number of topics
num_topics = 10

# Train the LDA model
lda_model = LdaModel(bow_corpus, num_topics=num_topics, id2word=dictionary, passes=10, random_state=100)

# Print the topics
print("LDA Topics:")
for idx, topic in lda_model.print_topics(-1):
    print(f"Topic: {idx}\nWords: {topic}\n")

LDA Topics:
Topic: 0
Words: 0.027*"series" + 0.025*"time" + 0.024*"attention" + 0.021*"feature" + 0.014*"temporal" + 0.013*"prediction" + 0.012*"information" + 0.010*"based" + 0.010*"proposed" + 0.010*"forecasting"

Topic: 1
Words: 0.049*"video" + 0.022*"frame" + 0.019*"motion" + 0.017*"flow" + 0.015*"action" + 0.014*"object" + 0.012*"recognition" + 0.011*"temporal" + 0.011*"based" + 0.011*"approach"

Topic: 2
Words: 0.023*"image" + 0.018*"generative" + 0.014*"face" + 0.012*"adversarial" + 0.011*"data" + 0.010*"user" + 0.010*"gan" + 0.009*"generation" + 0.008*"real" + 0.008*"generated"

Topic: 3
Words: 0.057*"image" + 0.012*"resolution" + 0.012*"depth" + 0.009*"based" + 0.008*"estimation" + 0.008*"segmentation" + 0.008*"approach" + 0.008*"shape" + 0.007*"result" + 0.007*"using"

Topic: 4
Words: 0.014*"deep" + 0.014*"neural" + 0.012*"machine" + 0.011*"decision" + 0.011*"tree" + 0.009*"explanation" + 0.008*"prediction" + 0.007*"data" + 0.006*"paper" + 0.006*"analysis"

Topic: 5
Words: 0.

In [ ]:
import pandas as pd

df1 = pd.read_csv(f'{path}/arxiv_data_210930-054931.csv')

corpus_nmf = df1['abstracts']
print(corpus_nmf.head())

0    Graph neural networks (GNNs) have been widely ...
1    Deep networks and decision forests (such as ra...
2    Graph convolutional networks (GCNs) are powerf...
3    With the increasing popularity of Graph Neural...
4    Machine learning solutions for pattern classif...
Name: abstracts, dtype: object


## Text Preprocessing for NMF


In [17]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Ensure NLTK data is downloaded (if not already present)
nltk.download('punkt', quiet=True)
nltk.download('stopwords', quiet=True)
nltk.download('wordnet', quiet=True)
nltk.download('omw-1.4', quiet=True)
nltk.download('punkt_tab', quiet=True) # Added to resolve LookupError

def clean_text(text):
    text = text.lower() # Convert to lowercase
    text = re.sub(r'\d+', '', text) # Remove numbers
    text = re.sub(r'[\W_]+', ' ', text) # Remove punctuation and special characters, replace with space
    text = re.sub(r'\s+', ' ', text).strip() # Remove extra spaces and strip leading/trailing spaces
    return text

stop_words = set(stopwords.words('english'))

def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return filtered_tokens

lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    lemmas = [lemmatizer.lemmatize(word) for word in tokens]
    return lemmas

cleaned_corpus_nmf = corpus_nmf.apply(clean_text)
tokenized_corpus_nmf = cleaned_corpus_nmf.apply(tokenize_and_remove_stopwords)
lemmatized_corpus_nmf = tokenized_corpus_nmf.apply(lemmatize_tokens)

# Join the lemmatized tokens back into single strings
preprocessed_corpus_nmf = lemmatized_corpus_nmf.apply(lambda x: ' '.join(x))

print("Original corpus (first 5):")
print(corpus_nmf.head())
print("\nCleaned corpus (first 5):")
print(cleaned_corpus_nmf.head())
print("\nTokenized corpus (first 5):")
print(tokenized_corpus_nmf.head())
print("\nLemmatized corpus (first 5):")
print(lemmatized_corpus_nmf.head())
print("\nPreprocessed corpus for NMF (first 5):")
print(preprocessed_corpus_nmf.head())

Original corpus (first 5):
0    Graph neural networks (GNNs) have been widely ...
1    Deep networks and decision forests (such as ra...
2    Graph convolutional networks (GCNs) are powerf...
3    With the increasing popularity of Graph Neural...
4    Machine learning solutions for pattern classif...
Name: abstracts, dtype: object

Cleaned corpus (first 5):
0    graph neural networks gnns have been widely us...
1    deep networks and decision forests such as ran...
2    graph convolutional networks gcns are powerful...
3    with the increasing popularity of graph neural...
4    machine learning solutions for pattern classif...
Name: abstracts, dtype: object

Tokenized corpus (first 5):
0    [graph, neural, networks, gnns, widely, used, ...
1    [deep, networks, decision, forests, random, fo...
2    [graph, convolutional, networks, gcns, powerfu...
3    [increasing, popularity, graph, neural, networ...
4    [machine, learning, solutions, pattern, classi...
Name: abstracts, dtype: object

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate CountVectorizer
vectorizer = CountVectorizer()

# Fit and transform the preprocessed corpus to create the Document-Term Matrix (DTM)
dtm = vectorizer.fit_transform(preprocessed_corpus_nmf)

print("Document-Term Matrix shape:", dtm.shape)
print("Number of unique terms (vocabulary size):", len(vectorizer.get_feature_names_out()))

Document-Term Matrix shape: (56181, 52271)
Number of unique terms (vocabulary size): 52271


In [20]:
from sklearn.decomposition import NMF

# Set the number of topics (e.g., 10 topics)
num_topics_nmf = 10

# Instantiate NMF model with increased max_iter
nmf_model = NMF(n_components=num_topics_nmf, random_state=100, init='nndsvda', max_iter=1000)

# Fit the NMF model to the DTM
nmf_model.fit(dtm)

print(f"NMF model fitted with {num_topics_nmf} components.")

NMF model fitted with 10 components.


In [21]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print(f"Topic #{topic_idx + 1}:")
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

n_top_words = 10
feature_names = vectorizer.get_feature_names_out()
print("NMF Topics:")
print_top_words(nmf_model, feature_names, n_top_words)

NMF Topics:
Topic #1:
learning task algorithm deep representation policy reinforcement agent problem approach
Topic #2:
image segmentation resolution color using quality result task adversarial approach
Topic #3:
graph node representation structure gnns information edge task learning classification
Topic #4:
model based prediction performance show attention generative trained training time
Topic #5:
object detection detector scene point dataset box task segmentation performance
Topic #6:
network neural deep architecture convolutional training layer performance result accuracy
Topic #7:
data time training domain series real approach set using distribution
Topic #8:
video frame temporal motion flow action approach time task optical
Topic #9:
feature attention information representation module point multi local level proposed
Topic #10:
method based proposed problem point state result propose algorithm art

